# Binairo 

Binairo is played on a rectangular grid with no standard size. Some cells start out filled with black or white circles. The rest of the cells are empty. The goal is to place circles in all cells in such a way that:

1. Each row and each column must contain an equal number of white and black circles.

2. More than two circles of the same color can't be adjacent.

3. Each row and column is unique. 

[Play On Janko](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/index.htm)

---------

1. Binairo 在矩形网格上播放，没有标准尺寸。有些单元格开始时充满了黑色或白色的圆圈。其余的单元格是空的。目标是以以下方式在所有单元格中放置圆圈；
2. 每行和每列必须包含相同数量的白色和黑色圆圈。
3. 两个以上相同颜色的圆圈不能相邻。
4. 每一行每一列都是独一无二的。

![](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Tohu-Wa-Vohu/Regeln-02.gif)

In [1]:
def readGrid(path):
    with open(f"../assets/data/Binairo/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("131_8x8")
    print(m, n)
    for g in grid:
        print(g)

8 8
['-', '2', '2', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '1', '-', '-', '2']
['-', '-', '1', '-', '1', '1', '-', '-']
['-', '-', '-', '-', '-', '1', '1', '-']
['-', '-', '1', '-', '-', '-', '-', '-']
['-', '2', '-', '-', '-', '-', '-', '1']
['-', '-', '-', '1', '2', '1', '-', '2']
['1', '-', '-', '-', '1', '1', '-', '-']


In [2]:
from ortools.sat.python import cp_model as cp

class BinarioSolver:
    
    def __init__(self, X, Y, grid) -> None:
        self.X = X
        self.Y = Y
        self.grid = grid 
        self.x = {}
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        
        # --- 改进 1: 统一将所有格子初始化为变量 ---
        for i in range(self.X):
            for j in range(self.Y):
                self.x[i, j] = self.model.NewBoolVar(f'x[{i}, {j}]')
                
                # 添加预填值的约束
                if self.grid[i][j] == "1": # 假设 1 代表白色(0)
                    self.model.Add(self.x[i, j] == 0)
                elif self.grid[i][j] == "2": # 假设 2 代表黑色(1)
                    self.model.Add(self.x[i, j] == 1)
                # "-" 保持自由变量

    def _add_distinct_vectors(self, vector_list):
        """
        通用函数：接收一组向量（列表的列表），确保其中任意两个向量不完全相同。
        实现逻辑：Sum(|v1[k] - v2[k]|) >= 1
        """
        num_vectors = len(vector_list)
        vec_len = len(vector_list[0])

        # 两两比较
        for i in range(num_vectors):
            for j in range(i + 1, num_vectors):
                diffs = []
                for k in range(vec_len):
                    # 定义一个布尔变量表示该位是否不同
                    # diff_var = 1 if vec_i[k] != vec_j[k] else 0
                    diff_var = self.model.NewBoolVar(f'diff_{i}_{j}_{k}')
                    
                    # 利用 AbsEquality capturing 差的绝对值
                    # 因为是 0/1 变量，差的绝对值即异或(XOR)逻辑
                    self.model.AddAbsEquality(diff_var, vector_list[i][k] - vector_list[j][k])
                    diffs.append(diff_var)
                
                # 约束：两个向量所有的差异位之和至少为 1
                self.model.Add(sum(diffs) >= 1)

    def addConstr(self):
        # 1. 连续不超过2个相同颜色的约束
        for i in range(self.X):
            for j in range(self.Y - 2):
                arr = [self.x[i, y] for y in range(j, j + 3)]
                self.model.Add(sum(arr) <= 2)
                self.model.Add(sum(arr) >= 1)

        for j in range(self.Y):
            for i in range(self.X - 2):
                arr = [self.x[xx, j] for xx in range(i, i + 3)]
                self.model.Add(sum(arr) <= 2)
                self.model.Add(sum(arr) >= 1)
        
        # 2. 行列黑白数量相等 (0和1数量相等)
        for i in range(self.X):
            self.model.Add(sum(self.x[i,j] for j in range(self.Y)) == int(self.Y) // 2)
        for j in range(self.Y):
            self.model.Add(sum(self.x[i,j] for i in range(self.X)) == int(self.X) // 2)

        # --- 改进 2: 添加唯一性约束 ---
        # 提取所有行
        rows = [[self.x[i, j] for j in range(self.Y)] for i in range(self.X)]
        self._add_distinct_vectors(rows)

        # 提取所有列
        cols = [[self.x[i, j] for i in range(self.X)] for j in range(self.Y)]
        self._add_distinct_vectors(cols)
    
    def printgrid(self):
        if self.status != cp.OPTIMAL and self.status != cp.FEASIBLE:
            print("No Solution Found")
            return
            
        for i in range(self.X):
            for j in range(self.Y):
                # 原始网格是为了视觉参考，这里建议直接打印解的结果
                val = self.solver.Value(self.x[i, j])
                if val == 1:
                    print("2", end=" ") # 对应输入数据里的 "2"
                else:
                    print("1", end=" ") # 对应输入数据里的 "1"
            print()
        print()
    
    def solve(self):
        self.status = self.solver.Solve(self.model)
        print(f"Status: {self.solver.StatusName(self.status)}")
        self.printgrid()

# --- 模拟测试 ---
def mock_read_and_run():
    # 模拟一个小的 6x6 Binairo 题目
    # 1=White(0), 2=Black(1), -=Empty
    raw_grid = [
        "- 1 - - - -",
        "- - - 2 - -",
        "- 1 - - - 1",
        "- - - - - -",
        "2 - - 2 - -",
        "- - - - - -"
    ]
    grid = [r.split(" ") for r in raw_grid]
    m, n = 6, 6 # 必须是偶数
    
    solver = BinarioSolver(m, n, grid)
    solver.addConstr()
    solver.solve()

if __name__ == "__main__":


    x, y, grid = readGrid("131_8x8")
    BinarioSolverTest = BinarioSolver(x, y, grid)
    BinarioSolverTest.addConstr()
    BinarioSolverTest.solve()
                
# 1 2 2 1 2 2 1 1 
# 2 1 1 2 1 2 1 2 
# 2 2 1 2 1 1 2 1 
# 1 2 2 1 2 1 1 2 
# 2 1 1 2 1 2 2 1 
# 2 2 1 1 2 2 1 1 
# 1 1 2 1 2 1 2 2 
# 1 1 2 2 1 1 2 2 

Status: OPTIMAL
1 2 2 1 2 2 1 1 
2 1 1 2 1 2 1 2 
2 2 1 2 1 1 2 1 
1 2 2 1 2 1 1 2 
2 1 1 2 1 2 2 1 
2 2 1 1 2 2 1 1 
1 1 2 1 2 1 2 2 
1 1 2 2 1 1 2 2 

